In [ ]:
import os
from pprint import pprint
import pandas as pd
pd.set_option('display.max_colwidth', -1)

DATA_DIR = '/Users/singhn4/Projects/kids_first/data/Schiffman'

In [ ]:
# Helper functions
def dropna_rows_cols(df_func):
    """
    Decorator to drop rows and cols w all nan values
    """

    def wrapper(*args, **kwargs):
        df = df_func(*args, **kwargs)

        # None or empty df
        try:
            if df.empty:
                return df
        except AttributeError:
            return df

        # Rows
        df.dropna(how="all", inplace=True)
        # Cols
        df.dropna(how="all", axis=1, inplace=True)
        return df

    return wrapper

def reformat_column_names(df_func):
    """
    Decorator to reformat DataFrame column names.

    Replace all column names having whitespace with underscore
    and make lowercase
    """

    def wrapper(*args, **kwargs):
        df = df_func(*args, **kwargs)
        # None or empty df
        try:
            if df.empty:
                return df
        except AttributeError:
            return df
        df.columns = map((lambda x: x.replace(" ", "_").lower()),
                         df.columns)
        return df

    return wrapper

In [ ]:
# Create study
study = {
    'data_access_authority': 'dbGaP',
    'study_id': 'phs001194',
    'study_version': 'v1.p2',
    'study_name': 'Expanded Ewing sarcoma cohort for tumor genomics and association with DNA repair deficiences, clinical presentation, and outcome',
    'attribution': 'https://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/GetAcknowledgementStatement.cgi?study_id=phs001194.v1.p2'
}
study_df = pd.DataFrame([study])
study_df.to_csv(os.path.join(DATA_DIR, 'study.txt'))

# Create investigator
invest = {
    'investigator_name': 'Joshua Schiffman',
    'institution': 'University of Utah'
}
inv_df = pd.DataFrame([invest])
inv_df.to_csv(os.path.join(DATA_DIR, 'investigator.txt'))

In [ ]:
# Read study files data
@reformat_column_names
@dropna_rows_cols
def read_study_file_data(filepaths=None):
    """
    Read in raw study files
    """
    if not filepaths:
        filepaths = os.listdir(DATA_DIR)

    study_files = [{"study_file_name": f}
                   for f in filepaths]
    return pd.DataFrame(study_files)

In [ ]:
@reformat_column_names
@dropna_rows_cols
def read_study_data(filepath=None):
    """
    Read study data
    """
    if not filepath:
        filepath = os.path.join(DATA_DIR,
                                'study.txt')
    df = pd.read_csv(filepath)

    return df

In [ ]:
@reformat_column_names
@dropna_rows_cols
def read_investigator_data(filepath=None):
    """
    Read investigator data
    """
    if not filepath:
        filepath = os.path.join(DATA_DIR,
                                'investigator.txt')
    df = pd.read_csv(filepath)

    return df

In [ ]:
def cols_to_lower(df):
    df.columns = map((lambda x: x.replace(" ", "_").lower()), df.columns)

In [ ]:
@reformat_column_names
@dropna_rows_cols
def read_participant_data(filepath=None):
    if not filepath:
        filepath = os.path.join(DATA_DIR, 'Schiffman_X01 Sample List.xlsx')
    # Participants
    df = pd.read_excel(filepath)
    cols_to_lower(df)
    # Extract participant columns
    participant_df = df[['individual_name', 'ewing_trio_number', 'relationship_to_proband']]

    # Create is_proband col
    def func(row): return row.relationship_to_proband == 'Self/Case'
    participant_df['relationship_to_proband'] = participant_df.apply(func, axis=1)

    # Create family_id column
    participant_df.rename(columns={'ewing_trio_number': 'family_id'})
    
    return participant_df

In [ ]:
# Participants
filepath = os.path.join(DATA_DIR, 'Schiffman_X01 Sample List.xlsx')
all_data_df = pd.read_excel(filepath)
cols_to_lower(all_data_df)

def create_diagnosis_df(df):
    # Create diagnosis df
    df.loc[(df['morphology'] == "9260/3: Ewing's sarcoma\r\nEwing's tumor"),'morphology']= 'Ewing Sarcoma'    
    def func(row): return "_".join(['diagnosis', str(row.name)])
    df['diagnosis_id'] = df.apply(func, axis=1)
    df = df[['individual_name', 'age_at_diagnosis_(days)', 'morphology']]
    return df

In [ ]:
# Sequencing experiment
@reformat_column_names
@dropna_rows_cols
def read_genomic_data(filepath=None):
    if not filepath:
        df = pd.read_excel(os.path.join(DATA_DIR, 'Schiffman_EwingSarcoma_QC_vs_Phenotype.xlsx'))
        del df['instrument_data_ids']
    return df

def create_seq_exp_data(df):
    df = df[['build_id', 'mean_insert_size', 'pf_reads', 'phenotype_sheet_sample_name']]
    return df

In [ ]:
# Genomic files
import json 
df = read_genomic_data()
df.describe(include=['O']).T.sort_values('unique', ascending=False)

In [ ]:
# Create genomic file df
def create_genomic_file_df(df):
    df = df[['build_id', 'bam_path']]
    df['file_name'] = df['bam_path'].apply(lambda p: os.path.basename(p))

    def get_ext(fp):
        filename = os.path.basename(fp)
        parts = filename.split('.')
        if len(parts) > 2:
            ext = '.'.join(parts[1:])
        else:
            ext = parts[-1]
        return ext

    with open(os.path.join(DATA_DIR, 'genomic_file_uuid.json'), 'r') as json_file:
        uuid_dict = json.load(json_file)

    gf_dicts = []
    for k, v in uuid_dict.items():
        file_info = {
            'uuid': v['did'],
            'md5sum': v['hashes']['md5'],
            'file_url': v['urls'][0],
            'data_type': 'submitted aligned reads',
            'file_format': get_ext(v['urls'][0]),
            'file_name': os.path.basename(v['urls'][0])
        }
        gf_dicts.append(file_info)

    gf_df = pd.DataFrame(gf_dicts)

    genomic_file_df = pd.merge(df, gf_df, on='file_name')
    
    return genomic_file_df


### Explore

In [ ]:
pprint([f for f in os.listdir(DATA_DIR)])

#### Sample List (Schiffman_X01 Sample List.xlsx)

In [ ]:
sample_list_df = pd.read_excel(os.path.join(DATA_DIR, 'Schiffman_X01 Sample List.xlsx'))
cols_to_lower(sample_list_df)
print(sample_list_df.shape)
sample_list_df.head()

In [ ]:
sample_list_df.groupby('relationship_to_proband').describe()

In [ ]:
# Unique morphologies
print('Morphologies')
pprint(sample_list_df['morphology'].unique().tolist())
print('\nTopographies')
# Unique topographies
pprint(sample_list_df['topography'].unique().tolist())

#### QC Phenotype (Schiffman_EwingSarcoma_QC_vs_Phenotype.xlsx)

In [ ]:
qc_phenotype_df = pd.read_excel(os.path.join(DATA_DIR, 'Schiffman_EwingSarcoma_QC_vs_Phenotype.xlsx'))
del qc_phenotype_df['instrument_data_ids']
cols_to_lower(qc_phenotype_df)
qc_phenotype_df.head()

In [ ]:
qc_phenotype_df.describe(include=['O']).T.sort_values('unique', ascending=False)
# qc_phenotype_df.dtypes

#### Trio Summary

In [ ]:
trio_df = pd.read_excel(os.path.join(DATA_DIR, 'Schiffman_EwingSarcoma_QC_vs_Phenotype.xlsx'), sheet_name=1)
trio_df.head()

### Extract

In [ ]:
# Study
study_df = read_study_data()
study_df.head()

In [ ]:
# Investigator
investigator_df = read_investigator_data()
investigator_df.head()

In [ ]:
# Study files
study_file_df = read_study_file_data()
study_file_df.head()

In [ ]:
# Participants
participant_df = read_participant_data()
participant_df.head()

In [ ]:
# Diagnosis
diagnosis_df = create_diagnosis_df(all_data_df)
diagnosis_df.head()
diagnosis_df = diagnosis_df.where((pd.notnull(diagnosis_df)), None)
diagnosis_df.head()

In [ ]:
# Phenotypes
def create_phenotype_df(df):
    df['phenotype'] = "Ewing's Sarcoma"
    df['hpo_id'] = "HP:0012254"
    df['observed'] = pd.notnull(df['morphology'])
    return df
phenotype_df = create_phenotype_df(diagnosis_df)
phenotype_df.head()

In [ ]:
# Family relationships
df = all_data_df[['individual_name', 'relationship_to_proband', 'ewing_trio_number']]
df.drop_duplicates('individual_name', inplace=True)
df['ewing_trio_number'] = df['ewing_trio_number'].astype(str)
fams = []
# For each group
for name, g in df.groupby('ewing_trio_number'):
    fam = {}
    # For each individual in family group
    for idx, row in g.iterrows():
        rel = row['relationship_to_proband']
        if rel == 'Self/Case':
            col = 'relative_id'
        else:
            col = 'participant_id'
        fam[col] = row['individual_name']
        fam['participant_to_relative_relation'] = rel
    if rel != 'Self/Case':    
        fams.append(fam)
df = pd.DataFrame(fams)
def func(row): return "_".join(['rel', str(row.name)])
df['rel_id'] = df.apply(func, axis=1)

In [ ]:
# Create family relationships
df = all_data_df[['individual_name', 'relationship_to_proband', 'ewing_trio_number']]
family_dict = {}
for idx, row in df.iterrows():
    fam_id = row['ewing_trio_number']
    if fam_id not in family_dict:
        family_dict[fam_id] = {}
    family_dict[fam_id][row['relationship_to_proband']] = row['individual_name']
df = pd.DataFrame(list(family_dict.values()))
df